<img src =https://i.pinimg.com/474x/4f/72/d3/4f72d331372ed6a69e2b2fa3dce89645.jpg width=300>



### Análise de Dados - URBS Curitiba

### Autora: Caroline Attilio de Lima


# Importando bibliotecas e base de dados

In [1]:
import pandas as pd 
import numpy as np 
import json as json
import requests as requests
import re

In [2]:
# IMPORTANDO AS BASES DE DADOS - LINHAS

dfLinhas2021 = pd.read_json ('2021_01_01_linhas.json')

In [3]:
# IMPORTANDO AS BASES DE DADOS - PONTOS

dfPontos2021 = pd.read_json ('2021_01_01_pontosLinha.json')
getPontos2021 = pd.read_csv('data_r.csv')

In [8]:
def formatacao_df(df):
    df['LAT'] = df['LAT'].str.replace(',', '.')
    df['LON'] = df['LON'].str.replace(',', '.')
    df['LAT'] = df['LAT'].astype(float)
    df['LON'] = df['LON'].astype(float)
    df['LAT'] = round(df['LAT'],6)
    df['LON'] = round(df['LON'],6)
    return df

In [4]:
print('Número de linhas em 01 de janeiro de 2021:',dfLinhas2021['COD'].nunique())

Número de linhas em 01 de janeiro de 2021: 308


## Análise das linhas 

In [5]:
# DADOS DAS LINHAS 01/01/2021

dfLinhas2021.CATEGORIA_SERVICO.value_counts()

ALIMENTADOR        144
CONVENCIONAL        72
LINHA DIRETA        28
MADRUGUEIRO         19
TRONCAL             17
EXPRESSO            11
INTERBAIRROS         9
LIGEIRÃO             3
EVENTO ESPECIAL      2
CIRCULAR CENTRO      2
JARDINEIRA           1
Name: CATEGORIA_SERVICO, dtype: int64

## ANÁLISE DOS TIPOS DOS PONTOS DE ÔNIBUS

In [6]:
# REMOVENDO OS PONTOS DE ÔNIBUS DUPLICADOS

dfPontosUnicos2021 = dfPontos2021.drop_duplicates(subset='NUM', keep='first')

In [9]:
dfPontosUnicos2021 = formatacao_df(dfPontosUnicos2021)

<ipython-input-8-1a37ac58ca08>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LAT'] = df['LAT'].str.replace(',', '.')
<ipython-input-8-1a37ac58ca08>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LON'] = df['LON'].str.replace(',', '.')
<ipython-input-8-1a37ac58ca08>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [10]:
dfPontosUnicos2021.head()

,NOME,NUM,LAT,LON,SEQ,GRUPO,SENTIDO,TIPO,ITINERARY_ID,COD
0,"R. Prefeito Rosalvo G Mello Leitão, 101 - Cent...",120113,-25.414184,-49.265917,2,,Horário,Novo mobiliário,481,010
1,"R. Carlos Pioli, 471 - Bom Retiro",120110,-25.410517,-49.276479,42,,Horário,Novo mobiliário,481,010
2,"Rua Alferes Ângelo Sampaio, 1177 - Batel",180036,-25.442824,-49.282907,23,,Anti-horário,Novo mobiliário,482,011
3,"Rua Alferes Ângelo Sampaio, 615 - Água Verde",150220,-25.446854,-49.280945,24,,Anti-horário,Novo mobiliário,482,011
4,"R. José de Alencar, 361 - Alto da Rua Quinze",110027,-25.431271,-49.254113,40,,Anti-horário,Novo mobiliário,482,011


In [11]:
# DADOS TRATADOS NO R PARA PEGAR O BAIRRO DEVIDO A PROBLEMA DE INSTALAÇÃO COM O GEOPANDAS

getPontos2021.drop(columns=['x','bairro','grupo'],inplace=True)

In [12]:
# RENOMEANDO AS COLUNAS

getPontos2021 = getPontos2021.rename(columns={'nome.x':'endereço','nome.y':'bairro'})

In [15]:
getPontos2021

,endereço,num,seq,sentido,tipo,itinerary_id,cod,bairro,lon,lat
0,"R. Prefeito Rosalvo G Mello Leitão, 101 - Cent...",120113,2,Horário,Novo mobiliário,481,010,CENTRO CÍVICO,-49.265917,-25.414184
1,"R. Carlos Pioli, 471 - Bom Retiro",120110,42,Horário,Novo mobiliário,481,010,BOM RETIRO,-49.276479,-25.410517
2,"Rua Alferes Ângelo Sampaio, 1177 - Batel",180036,23,Anti-horário,Novo mobiliário,482,011,BATEL,-49.282907,-25.442824
3,"Rua Alferes Ângelo Sampaio, 615 - Água Verde",150220,24,Anti-horário,Novo mobiliário,482,011,ÁGUA VERDE,-49.280945,-25.446854
4,"R. José de Alencar, 361 - Alto da Rua Quinze",110027,40,Anti-horário,Novo mobiliário,482,011,ALTO DA RUA XV,-49.254113,-25.431271
...,...,...,...,...,...,...,...,...,...,...
7098,"Rua Júlio Wischral, 355 - Uberaba",140674,35,Vila Macedo,Sem demarcação,11595,477,UBERABA,-49.225821,-25.473994
7099,"Rua Francisco Nunes, 255 - Rebouças",140938,8,Vila Macedo,Domus,11595,477,REBOUÇAS,-49.256293,-25.443999
7100,"Rua Doutor Joaquim Ignácio Silveira da Motta, ...",140714,26,Vila Macedo,Placa em poste,11595,477,GUABIROTUBA,-49.239987,-25.460323
7101,"Rua Doutor Joaquim Ignácio Silveira da Motta, ...",140719,28,Vila Macedo,Sem demarcação,11595,477,GUABIROTUBA,-49.236996,-25.464393


In [26]:
# getPontos2021.to_csv('getPontos2021v2.csv')

In [16]:
getQtdPontos = getPontos2021.tipo.value_counts()
getQtdPontos

Chapéu chinês           2185
Novo mobiliário         1993
Domus                    762
Sem demarcação           587
Placa em cano            445
Plataforma               313
Placa em poste           294
Estação tubo             288
Especial Madrugueiro     188
Linha Turismo             23
Fora do padrão            13
Concreto                   9
Outros                     1
Name: tipo, dtype: int64

In [17]:
getQtdPontosBairro = getPontos2021.bairro.value_counts()

In [18]:
# QUANTIDADE DE TIPO DE PONTO POR BAIRRO

qtd_pontos = getPontos2021.groupby(['bairro','tipo']).agg({'tipo':'count'}) 

In [19]:
qtd_pontos.head()

tipo
bairro    tipo                 
ABRANCHES Chapéu chinês      36
          Domus              17
          Linha Turismo       1
          Novo mobiliário    10
          Placa em cano       7

In [20]:
# QUANTIDADE DE TIPO DE PONTO POR BAIRRO

qtd_tipos = getPontos2021.groupby(['tipo','bairro']).agg({'tipo':'count'})
qtd_tipos.head()

tipo
tipo          bairro              
Chapéu chinês ABRANCHES         36
              AHÚ                5
              ALTO BOQUEIRÃO    34
              ALTO DA GLÓRIA     4
              ALTO DA RUA XV     2

In [109]:
qtd_tipos = qtd_tipos.rename(columns={'tipo':'Tipo','bairro':'Bairro','tipo':'Qdt'}).reset_index()
qtd_tipos.head()

,tipo,bairro,Qdt
0,Chapéu chinês,ABRANCHES,36
1,Chapéu chinês,AHÚ,5
2,Chapéu chinês,ALTO BOQUEIRÃO,34
3,Chapéu chinês,ALTO DA GLÓRIA,4
4,Chapéu chinês,ALTO DA RUA XV,2


In [115]:
# Quais são os cinco bairros que mais contém pontos de ônibus sem demarcação?

getSemDemarcacao = qtd_tipos[qtd_tipos['tipo']=='Sem demarcação']
getSemDemarcacao.sort_values('Qdt',ascending=False).head(5)

,tipo,bairro,Qdt
448,Sem demarcação,CIDADE INDUSTRIAL DE CURITIBA,62
482,Sem demarcação,UBERABA,37
470,Sem demarcação,SANTA CÂNDIDA,34
442,Sem demarcação,CAMPO DE SANTANA,21
429,Sem demarcação,ALTO BOQUEIRÃO,19


In [116]:
# Quais são os três bairros com maior quantidade de estações tubo?

getEstTubo = qtd_tipos[qtd_tipos['tipo']=='Estação tubo']
getEstTubo.sort_values('Qdt',ascending=False).head(3)

,tipo,bairro,Qdt
178,Estação tubo,CENTRO,36
196,Estação tubo,REBOUÇAS,19
162,Estação tubo,ALTO BOQUEIRÃO,15


In [32]:
# # SALVANDO OS ARQUIVOS

# qtd_pontos.to_csv('qtd_pontos.csv')
# getQtdPontosBairro.to_csv('getQtdPontosBairro.csv')
# getQtdPontos.to_csv('getQtdPontos.csv')
# getPontos2021.to_csv('getPontos2021.csv')